# Baysian Networks
with R kernel


Bnlearn Tutorial: https://jacintoarias.github.io/bayesnetRtutorial/#the_bnlearn_package

In [103]:
list_of_packages <- c ("synthpop", "insight", "party", "dplyr", "rpart", "rpart.plot", "randomForest", "pROC", "caret", "pracma", "bnlearn", "arulesCBA")

lapply(list_of_packages, FUN= function(X){
  do.call("require", list(X))
})

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

# Data

In [104]:
load("/Users/emmafoessing/Documents/Master/MA/Synthpop/cpspop.RData")
adult <- read.csv("/Users/emmafoessing/Documents/Master/MA/Code/adult_preprocessed.csv")
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

adult_with_cont <- adult
cps_with_cont <- cpspop

In [135]:
class(adult$hours_per_week)

[1] "integer"

In [109]:
sum(adult_with_cont$capital_gain==0)
dim(adult_with_cont)

[1] 27624

[1] 30162    14

In [72]:
freq_table <- table(cpspop$csp)
length(cpspop)/2

In [83]:
vec <- c()
length(vec)

[1] 0

In [75]:
names(freq_table)[which(table(cpspop$csp) > length(cpspop$csp) / 2)]

[1] "0"

Problem, das es zu lösen gilt: <br>
Ich will meine kategorialen variables in meinem Datensatz diskretisieren (in Intervalle einteilen) --> das geht mit der discretize function <br>
Ich will ca. 5 Kategorien maximal pro Variable haben. Wenn eine der Ausprägungen in der Varibale mindestens 1/5 aller Ausprägungnen aus macht, dann soll diese eine eigene Kategorie werden und die restlichen als Interval kategorisiert werden. Wenn es mehrere Werte gibt, die mindestens 1/5 aller Ausprägungen ausmachen, dann sollen auch diese alle jeweils eine eigene Kategorie werden und der Rest kann in Intervalle eingeteilt werden. Am Ende soll die Variable des Datensatzes überschrieben werden mit der kategorialen Variable und der Datensatz nur aus 'factor' Variablen bestehen.

In [143]:
# function to discretize the continuous variables

discretize_df = function(df, breaks = 5) {
  for (var in colnames(df)) {
    # if variable is not a factor, discretize it
    if (!is.factor(df[[var]])) {

      # Count the frequency of each unique value
      freq_table <- table(df[[var]])

      # Identify if there are values making up at least 1/5th of the observations
      common_value <- as.numeric(names(freq_table)[which(freq_table > length(df[[var]]) / 5)])

      if (length(common_value) > 0) {
        # If such a value exists, treat it separately
        # Discretize the other values into categories
        if (sum(df[[var]]==as.numeric(common_value))/length(df[[var]])>0.8){
          new_breaks = 1
        }
        else if (sum(df[[var]]==as.numeric(common_value))/length(df[[var]])>0.6) {
           new_breaks = 2
        }
        else {
           new_breaks = breaks - 1
        }
        df <- df %>% 
          mutate(!!sym(var) := ifelse(df[[var]] != common_value, 
                           discretize(df[[var]], method = "cluster", breaks = new_breaks),
                           df[[var]]))
        # Combine the common value and the discretized other values
        df[[var]] <- as.factor(df[[var]])
      } else {
        # Else, simply discretize the variable normally
        df[[var]] <- discretize(df[[var]], method = "cluster", breaks = breaks)
      }
    }
  }
  return(df)
}

Funktion von GPT4: (funktioniert nicht!)

In [155]:
discretize_df = function(df, breaks = 5) {
    for (var in colnames(df)) {
        # if variable is not a factor, discretize it
        if (!is.factor(df[[var]])) {
            # Count the frequency of each unique value
            freq_table <- table(df[[var]])

            # Identify if there are values making up at least 1/5th of the observations
            common_value <- as.numeric(names(freq_table)[which(freq_table > length(df[[var]]) / 5)])

            if (length(common_value) > 0) {
                # If common_value exists, make it as a separate category
                df[[var]][df[[var]] == common_value] <- paste0('common_value_', common_value)
                
                # Get remaining values (excluding common_value) to discretize
                rest_values <- df[[var]] != paste0('common_value_', common_value)
                
                # discretize the remaining values into a desired number of intervals
                df[[var]][rest_values] <- as.factor(discretize(denumeric(df[[var]][rest_values]), 
                                                              method = "cluster", 
                                                              breaks = breaks,
                                                              include.lowest=TRUE, 
                                                              labels = FALSE))
            } else {
                # if common_value does not exist discretize the variable straightforwardly 
                df[[var]] <- as.factor(discretize(df[[var]], method = "cluster", 
                                                  breaks = breaks, 
                                                  include.lowest=TRUE, 
                                                  labels = FALSE))
            }
        } 
        # Convert the variable to a factor
        df[[var]] <- as.factor(df[[var]])
    }
    return(df)
}

andere Lösung (noch nicht ausprobiert)

In [ ]:
# Laden von benötigten Libraries
library(dplyr)
library(forcats)

# Angenommen, df ist Ihr Datensatz und var Ihre Variable.
# Zuerst erzeugen wir eine temporäre Variable mit der Häufigkeit jeder Ausprägung.
df <- df %>%
  group_by(var) %>%
  mutate(freq = n())

# Dann erzeugen wir die Intervalle/Kategorien.
df <- df %>%
  ungroup() %>%
  mutate(var = ifelse(freq/nrow(df) >= 1/5, as.character(var), cut_number(var, 5))) %>%
  mutate(var = as.factor(var))

# Schleife, um diesen Vorgang für alle Spalten durchzuführen.
for(col in colnames(df)) {
  df <- df %>%
    group_by(!!sym(col)) %>%
    mutate(freq = n()) %>%
    ungroup() %>%
    mutate(!!sym(col) := ifelse(freq/nrow(df) >= 1/5, as.character(!!sym(col)), cut_number(!!sym(col), 5))) %>%
    mutate(!!sym(col) := as.factor(!!sym(col)))
}

# Der resultierende Datensatz df sollte nur in Kategorien gehaltene Spalten enthalten.

# Using a Baysian Network to generate synthetic data
baysian_net <- function(data){
    data <- discretize_df(data)
    print(sum(is.na(data)))
    bn <- hc(data)  # Learn structure using Hill-Climbing
    bn <- bn.fit(bn, data)  # Learn parameters using Maximum Likelihood Estimation

    # Generate Synthetic Data
    syn_data <- rbn(bn, length(data))  # Generate same number of obs as in the original data

    return(syn_data)
    }

# Model

Problem: Baysian networks require data structure with solely discrete variables!
--> exists in both CPS and Adult
For continuous data, common practice would be to first discretize the variables (convert the continuous data into bins or categories) and then apply Bayesian Network algorithms. However, the process of discretization must be done very thoughtfully as ill-advised discretization can lead to loss of crucial information.

In [141]:
adult_syn <- baysian_net(adult)

[1] 0


In [68]:
sum(cpspop$csp==0)

[1] 47764

In [145]:
cps_syn <- baysian_net(cps_with_cont)

[1] 0


In [156]:
cps <- discretize_df(cps_with_cont)
head(cps)

ERROR: Error in denumeric(df[[var]][rest_values]): konnte Funktion "denumeric" nicht finden


In [154]:
unique(cps$ss)

[1] 0                   [8.14e+03,1.25e+04) [1.25e+04,2.65e+04)
  [4] [3.02e+03,8.14e+03) 2130                1649               
  [7] 733                 [2.65e+04,5e+04]    2826               
 [10] 2646                1225                1746               
 [13] 1938                2929                1242               
 [16] 2700                1095                560                
 [19] 899                 2946                2106               
 [22] 954                 2377                2770               
 [25] 2856                2994                2797               
 [28] 533                 2184                2178               
 [31] 1506                2058                3000               
 [34] 534                 1722                2100               
 [37] 639                 2298                2484               
 [40] 1800                2880                1740               
 [43] 2397                2694                2964               
 [46] 1638                2358                1446               
 [49] 2679                1036                1260               
 [52] 2802                1251                965                
 [55] 881                 2208                2370               
 [58] 2032                2031                2094               
 [61] 879                 1145                947                
 [64] 1082                1662                1046               
 [67] 2000                2630                2778               
 [70] 928                 1440                1971               
 [73] 2622                2682                2634               
 [76] 1946                1115                1628               
 [79] 945                 1866                1352               
 [82] 384                 1326                2190               
 [85] 500                 833                 850                
 [88] 2491                1500                1626               
 [91] 755                 1848                1644               
 [94] 2616                1674                480                
 [97] 1120                1380                1516               
[100] 2274                1968                2728               
[103] 1742                2232                2346               
[106] 2200                1672                2800               
[109] 1940                2706                471                
[112] 1300                1324                1170               
[115] 458                 1782                1550               
[118] 1902                1271                1831               
[121] 2046                2115                1776               
[124] 1372                2838                2382               
[127] 626                 1428                2586               
[130] 1792                2010                2070               
[133] 2332                950                 2676               
[136] 2454                1100                2292               
[139] 1854                1678                1741               
[142] 2400                904                 1283               
[145] 1310                2424                1794               
[148] 2874                1650                978                
[151] 600                 2850                1914               
[154] 2988                2118                2173               
[157] 800                 360                 558                
[160] 933                 2502                731                
[163] 535                 1000                1030               
[166] 1926                2695                2580               
[169] 2496                1400                1416               
[172] 1212                1608                2442               
[175] 1218                2662                266                
[178] 2034                845                 2038               
[181] 1154                1254            

In [147]:
unique(cps$tax)

[1] 0 1 2 3 4
Levels: 0 1 2 3 4